# 3. Machine Learning for Classification

We'll use logistic regression to predict churn

## 3.1 Churn Prediction Project

- Dataset: https://www.kaggle.com/blastchar/telco-customer-churn
- https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv

## 3.2 Data Preparation

- Download the data, read it with pandas
- Look at the data
- Make column names and values look uniform
- Check if all the columns read correctly
- Check if the churn variable needs any preparation

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Data url
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv'

In [3]:
# If data is not in the directory
# download it as 'churn-data.csv'
if not os.path.isfile('churn-data.csv'):
    !wget $data -O churn-data.csv

In [4]:
df = pd.read_csv('churn-data.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [5]:
# Transpose df to view all columns
df.head().T

,0,1,2,3,4
customerID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
tenure,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No


In [6]:
# Make columns names and values look uniform
df.columns = df.columns.str.lower().str.strip()

categorical_cols = df.select_dtypes(include='object').columns.tolist()

for col in categorical_cols:
    df[col] = df[col].str.lower().str.replace(' ', '_')

In [7]:
df.head().T

,0,1,2,3,4
customerid,7590-vhveg,5575-gnvde,3668-qpybk,7795-cfocw,9237-hqitu
gender,female,male,male,male,female
seniorcitizen,0,0,0,0,0
partner,yes,no,no,no,no
dependents,no,no,no,no,no
tenure,1,34,2,45,2
phoneservice,no,yes,yes,no,yes
multiplelines,no_phone_service,no,no,no_phone_service,no
internetservice,dsl,dsl,dsl,dsl,fiber_optic
onlinesecurity,no,yes,yes,yes,no


In [8]:
# Check datatypes of all columns
df.dtypes

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges         object
churn                object
dtype: object

In [9]:
# Convert data type of 'totalcharges' from string to numeric
# pass errors='coerce' to set not parsable values to null
df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')

In [10]:
# Check missing values in 'totalcharges'
df.totalcharges.isnull().sum()

11

In [11]:
# Fill missing values of 'totalcharges' with 0
df.totalcharges = df.totalcharges.fillna(0)

# Check missing values again
df.totalcharges.isnull().sum()

0

In [12]:
# View 'churn' values
df.churn.head()

0     no
1     no
2    yes
3     no
4    yes
Name: churn, dtype: object

In [13]:
# Convert 'churn' values to int
df.churn = (df.churn == 'yes').astype(int)

## 3.3 Setting Up the Validation Framework

In [16]:
from sklearn.model_selection import train_test_split

# Split train (80%), test (20%)
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
# Split train (60%), validation (20%)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [17]:
len(df_train), len(df_val), len(df_test)

(4225, 1409, 1409)

In [18]:
# Reset index
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [20]:
# Create target variable 'churn' for train, validation, test sets
y_train = df_train.churn.values
y_val = df_val.churn.values
y_test = df_test.churn.values

In [21]:
# Delete target varaible 'churn' from train, validation, test sets
del df_train['churn']
del df_val['churn']
del df_test['churn']

## 3.4 EDA

- Check missing values
- Look at the target variable (churn)
- Look at numerical and categorical variables

In [22]:
# Reset index of 'df_full_train'
df_full_train = df_full_train.reset_index(drop=True)

In [23]:
# Check missing values
df_full_train.isnull().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [24]:
# Check distribution of the target vairable 'churn'
# normalize=True will return the percentage of the distribution
df_full_train.churn.value_counts(normalize=True)

0    0.730032
1    0.269968
Name: churn, dtype: float64

When we calculate the mean of a binary column, when get the mean which is the fraction of 1's, since the column only consists of values 0's and 1's.

In [25]:
# Calculate the mean of churn, is this called 'global churn rate'
df_full_train.churn.mean()

0.26996805111821087

In [26]:
# Check the data types of the columns in 'df_full_train'
df_full_train.dtypes

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges        float64
churn                 int32
dtype: object

Note: column `seniorcitizen` is a categorical column not the numerical because it consists of values 0 and 1. The numerical columns are `tenure`, `monthlycharges`, and `totalcharges`.

In [27]:
# Create a list of numerical cols
numerical_cols = ['tenure', 'monthlycharges', 'totalcharges']

In [33]:
# Create a list of categorical cols
categorical_cols = ['gender', 'seniorcitizen', 'partner',
                    'dependents', 'phoneservice', 'multiplelines',
                    'internetservice', 'onlinesecurity', 'onlinebackup',
                    'deviceprotection', 'techsupport', 'streamingtv',
                    'streamingmovies', 'contract', 'paperlessbilling',
                    'paymentmethod']

In [34]:
# Find the number of unique values in 'categorical_cols'
df_full_train[categorical_cols].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

## 3.5 Feature Importance: Chrun Rate and Risk Ratio

Feature importance analysis (part of EDA) - identifying which features affect our target vairable.

- Churn rate
- Risk ratio
- Mutual information - later

### Churn Rate

In [35]:
df_full_train.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,5442-pptjy,male,0,yes,yes,12,yes,no,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,19.70,258.35,0
1,6261-rcvns,female,0,no,no,42,yes,no,dsl,yes,...,yes,yes,no,yes,one_year,no,credit_card_(automatic),73.90,3160.55,1
2,2176-osjuv,male,0,yes,no,71,yes,yes,dsl,yes,...,no,yes,no,no,two_year,no,bank_transfer_(automatic),65.15,4681.75,0
3,6161-erdgd,male,0,yes,yes,71,yes,yes,dsl,yes,...,yes,yes,yes,yes,one_year,no,electronic_check,85.45,6300.85,0
4,2364-ufrom,male,0,no,no,30,yes,no,dsl,yes,...,no,yes,yes,no,one_year,no,electronic_check,70.40,2044.75,0


In [43]:
# Calculate global churn rate
global_churn = df_full_train.churn.mean()
global_churn

0.26996805111821087

In [44]:
# Calculate churn rate in 'female' gender
churn_female = df_full_train[df_full_train.gender == 'female'].churn.mean()
churn_female

0.27682403433476394

In [45]:
# Calculate the difference between 'female churn' and 'global churn' rate
churn_female - global_churn

0.006855983216553063

In [46]:
# Calculate churn rate in 'male' gender
churn_male = df_full_train[df_full_train.gender == 'male'].churn.mean()
churn_male

0.2632135306553911

In [47]:
# Calculate the difference between 'male churn' and 'global churn' rate
churn_male - global_churn

-0.006754520462819769

In [49]:
# Find distribution in 'partner'
df_full_train['partner'].value_counts()

no     2932
yes    2702
Name: partner, dtype: int64

In [50]:
# Calculate churn rate of people with partner
churn_partner = df_full_train[df_full_train.partner == 'yes'].churn.mean()
churn_partner

0.20503330866025166

In [51]:
# Calculate the difference between 'churn_partner' and 'global_churn' rate
churn_partner - global_churn

-0.06493474245795922

In [52]:
# Calculate churn rate of people without partner
churn_no_partner = df_full_train[df_full_train.partner == 'no'].churn.mean()
churn_no_partner

0.3298090040927694

In [53]:
# Calculate the difference between 'churn_no_partner' and 'global_churn' rate
churn_no_partner - global_churn

0.05984095297455855

### Risk Ratio

In [54]:
# Calculate the churn risk ratio of people without partner
churn_no_partner / global_churn

1.2216593879412643

In [55]:
# Calculate the churn risk ratio of people with partner
churn_partner / global_churn

0.7594724924338315

If we want to calculate the churn **difference** and **ratio** between `gender` in the `df_train_full` data using SQL, the query will look like this:

```sql
SELECT gender,
       AVG(churn),
       AVG(churn) - global_churn AS diff,
       AVG(churn) / global_churn AS risk
FROM data
GROUP BY gender;
```

We can write the above query in Pandas using `groupby()` method. 

In [65]:
# Groupby 'gender' to calculate the churn 'mean', 'count',
# 'difference' and 'risk'
df_group = df_full_train.groupby('gender')['churn'].agg(['mean', 'count'])
df_group['diff'] = df_group['mean'] - global_churn
df_group['risk'] = df_group['mean'] / global_churn
df_group

,mean,count,diff,risk
gender,,,,
female,0.276824,2796,0.006856,1.025396
male,0.263214,2838,-0.006755,0.974980


In [72]:
# Calculate the above aggregation for all the variables in
# 'df_full_train'
for c in categorical_cols:
    print(c)
    df_group = df_full_train.groupby(c)['churn'].agg(['mean', 'count'])
    df_group['diff'] = df_group['mean'] - global_churn
    df_group['risk'] = df_group['mean'] / global_churn
    display(df_group)

gender


,mean,count,diff,risk
gender,,,,
female,0.276824,2796,0.006856,1.025396
male,0.263214,2838,-0.006755,0.974980


seniorcitizen


,mean,count,diff,risk
seniorcitizen,,,,
0,0.242270,4722,-0.027698,0.897403
1,0.413377,912,0.143409,1.531208


partner


...

,mean,count,diff,risk
contract,,,,
month-to-month,0.431701,3104,0.161733,1.599082
one_year,0.120573,1186,-0.149395,0.446621
two_year,0.028274,1344,-0.241694,0.104730


paperlessbilling


,mean,count,diff,risk
paperlessbilling,,,,
no,0.172071,2313,-0.097897,0.637375
yes,0.338151,3321,0.068183,1.252560


paymentmethod


,mean,count,diff,risk
paymentmethod,,,,
bank_transfer_(automatic),0.168171,1219,-0.101797,0.622928
credit_card_(automatic),0.164339,1217,-0.105630,0.608733
electronic_check,0.455890,1893,0.185922,1.688682
mailed_check,0.193870,1305,-0.076098,0.718121


## 3.6 Feature Importance: Mutual Information